# 因子开发与回测示例

本 Notebook 展示了如何使用 `longgang_trader` 框架进行一次完整的因子开发和回测流程。

## 1. 加载数据

In [ ]:
from longgang_trader.data.data_loader import DataLoader

# 初始化数据加载器（此处应有配置文件）
dl = DataLoader(config={...})

# 加载数据
stock_data = dl.load_stock_data([...]) # type: ignore

## 2. 计算因子

In [ ]:
from longgang_trader.alpha.factor_calculator import MomentumFactor, FactorCalculator

# 创建一个20日动量因子
mom_factor = MomentumFactor(window=20)

# 使用因子计算器
calculator = FactorCalculator(stock_data)
calculator.add_factor(mom_factor)
factor_df = calculator.run()

## 3. 因子检验

In [ ]:
from longgang_trader.alpha.factor_testing import FactorTester

# 准备收益率数据
returns_data = ...

# 初始化因子检验器
tester = FactorTester(factor_df, returns_data, ...)

# 运行检验并查看结果
tester.run_all_tests()

## 4. 回测

In [ ]:
from longgang_trader.backtesting.backtester import Backtester, BaseStrategy

# 定义一个简单的分层回测策略
class SimpleLayeredStrategy(BaseStrategy):
    def generate_signals(self, current_date, current_positions):
        # 在每个调仓日，买入因子值最高的10%的股票，卖出最低的10%
        pass

# 初始化策略和回测器
strategy = SimpleLayeredStrategy(factor_df)
bt = Backtester(strategy, stock_data, initial_capital=10000000, transaction_cost=0.001)

# 运行回测
# 在运行前，请确保已经编译了Rust核心: `cargo build --release`
bt.run_backtest()

# 查看表现
bt.plot_equity_curve()
print(bt.calculate_performance_metrics())